# Control flow

Like all high-level languages, Julia provides a full set of modern control flow constructs:

- Compound Expressions: begin and ;
- Conditional Evaluation: if-elseif-else and ?: (ternary operator)
- Short-Circuit Evaluation: &&, || and chained comparisons
- Loops: while and for
- Exception Handling: try-catch, error and throw

## Compound expression

Compound expression is a sequence of subexpressions that are evaluated in order, its value is the the last subexpression.
It allows you to use multiple expressions where the syntax only allows one, for example, a one-statement function definition.

There are two varieties: `begin-end` block and `(;)` chains.

Their form are

```
begin
    <expr-1>
    ...
    <expr-n>
end
```

```
y = (expr-1; ...; expr-n>)
```

In [1]:
x = 3
begin
    intercept = 5
    slope = 3
    intercept + slope * x
end

14

The equivalent is:

In [2]:
z(x) = (slope = 3; intercept = 5; intercept + slope * x)

z (generic function with 1 method)

In [3]:
z(4)

17

## Three-valued relational logic

Julia's relational operators are as follows:

Expression | Name 
---------- | --------- 
~x | bitwise not
x & y | bitwise and
x &#124; y | bitwise or
x ⊻ y | bitwise xor (exclusive or)

Whereas comparison operators propagate missing values as it is not possible to compare a given value against something that is not given, 
logical operators are different, they obey [three-value logic](https://en.wikipedia.org/wiki/Three-valued_logic):

**Not**

A | ~A
--|------
false | true
missing | missing
true | false

**And**

A & B | B: false | B: missing | B: true
--|--------|--------|--------------
**A: false** | false | false | false
**A: missing** | false | missing | missing
**A: true** | false | missing | true

**Or**

A &#124; B | B: false | B: missing | B: true
--|--------|--------|--------------
**A: false** | false | missing | true
**A: missing** | missing | missing | true
**A: true** | true | true | true

**Xor**

A ⊻ B     | B: false | B: missing | B: true
--------|---------|-----------|--------------
**A: false**   | false   | missing   | true
**A: missing** | missing | missing   | missing
**A: true**    | true    | missing   | false

Note that these operators can operate on integers as bitwise operators:

In [4]:
~3            # not flips bits

-4

In [5]:
1 & 3         # bitwise and

1

In [6]:
1 | 3         # bitwise or

3

In [7]:
1 ⊻ 3         # bitwise xor

2

## if-elseif-else

The if-elseif-else blocks allow unlimited conditional evaluation of statements, the first block where the conditional test is true is evaluated, otherwise the else block is evaluated (if present).

```
if test1
    stmts        # multiple statements
elseif test2     # optional, can repeat many times
    stmts
else             # optional, executed when all prior tests fail
    stmts
end              # not optional, mandatory
```

In keeping with minimalist syntax, Julia doesn't use the word "then" and block delimiters such as "do; end;" or "{ }" are unnecessary.

In [8]:
x = 3; y = 4

if x < y
    r = "x less than y"
elseif x == y
    r = "x = y"
else
    r = "x larger than y"
end

println("x is $x, y is $y, $r")

x is 3, y is 4, x less than y


Unlike other languages, the if statement returns a value which is the return value of the last executed statement in the branch that was chosen (if any).
If all of the branches assign value to the same variable, it can look simpler:

In [9]:
x = 3; y = 3

r = if x < y
    "x less than y"
elseif x == y
    "x == y"
else
    "x larger than y"
end

println("x is $x, y is $y, $r")

x is 3, y is 3, x == y


## Ternary Operator

The ternary operator is a simple two-way alternative expression evaluation mechanism, ternary means an operator that takes three operands.

```
test ? true-expr : false-expr     # mandatory whitespace surrounding ? and :
```

The tokens ? and : must be surrounded by whitespaces, newline is acceptable.

If test is true, true-expr is evaluated, otherwise false-expr is evaluated.
Just like if-elseif-else, only one of the expressions is executed; this matters if the expressions have side-effects.

In [10]:
x = 1; y = 2
println(x < y ? "x < y" : "x >= y")

x < y


In [11]:
x = 2; y = 2
println(x < y ? "x < y" : "x >= y")

x >= y


While Julia allows chaining ternary operators to create more complex logic, they are not intuitive to read and are best avoided:

In [12]:
x = 2; y = 2
print(x < y ? "x < y" : 
      x > y ? "x > y" : "x == y")

x == y

## Short-Circuit Evaluation

Short-circuit evaluation is also known as **lazy evaluation**.
In a series of boolean expressions connected by && (and) and || (or) boolean operators, only the minimum number of expressions are evaluated as necessary.
As soon as the result can be ascertained, the rest of the boolean conditions are not evaluated.
These two operators can be used to put together a complex sequence of logic with the usual precedence order (&& before ||) and associativity (to the right).
All operands must evaluate to true-false-missing except the very last expression which may be evaluated and returned depending on the preceding conditionals.

In their simplest form,

- in the expression `a && b`, b is evaluated only if a is true. If a is false, the expression is false, there's no need to evaluate b.

- in the expression `a || b`, b is evaluated only if a is false. If a is true, the expression is true, there's no need to evaluate b.

They are often used as very short if statements to check conditions and assert values before execution:

- **`<cond> && <statement>`** is equivalent to `if <cond> <statement> end`. It is short form of if true < statement >.

- **`<cond> || <statement>`** is equivalent to `if ! <cond> <statement> end`. It is short form of if not true < statement >, or an assertion.

For example,

```
function factorial(n::Int)
    n >= 0 || error("n must be non-negative")    # or else:   check n >= 0 or else error
    n == 0 && return 1                           # and then:  if n == 0 and then return 1
    n * factorial(n-1)
end
```

## while

A while loop evaluates a condition and, as long as the condition is true, keeps executing the loop body.
Because the condition is tested at the top of the loop, it is possible the loop is not executed.
Here's an example:

In [13]:
i = 1                 # initialize
while i <= 3          # test
    println(i)        # body
    i += 1            # update
end

1
2
3


Julia does not have an until loop which is test at the bottom and the body is executes at least once.
But you can use this construct for the same effect:

```
test = true               # initialize to true
while test                # test
    <body>                # body
    <update test>         # update
end
```

## for

When the sequence to be iterated over is known or has a regular structure, a for loop is more idiomatic.
Iterating over a given sequence:

In [14]:
for i in [1, 3, 5, 7]
    println(i)
end

1
3
5
7


Iterating over an iterable:

In [15]:
for i in 1:3               # start:stop
    println(i)
end

1
2
3


In [16]:
for i in 1:2:7             # start:step:stop
    println(i)
end

1
3
5
7


## break and continue

With both the while loop and for loop, you can use **break** to exit the loop immediately or **continue** to skip the rest of the body:

In [17]:
for k in 1:10
    k <= 6 || break         # k <= 6 or else ...
    k == 3 && continue      # k == 3 and then ...
    println("k is $k")
end

k is 1
k is 2
k is 4
k is 5
k is 6


## Nested for loop

Loops can be nested, break/continue are associated with their respective loop:

In [18]:
for i = 1:2
    println("i is $i  -------")
    for j = 11:12
        println("i is $i  j is $j")
    end
end

i is 1  -------
i is 1  j is 11
i is 1  j is 12
i is 2  -------
i is 2  j is 11
i is 2  j is 12


Because multi-level nested loop occurs frequently in scientific computations, you can specify all the iterators in a single for loop:

In [19]:
for i = 1:2, j = 11:12, k = 101:102            # three level nested for loop
    println("i is $i   j is $j   k is $k")
end

i is 1   j is 11   k is 101
i is 1   j is 11   k is 102
i is 1   j is 12   k is 101
i is 1   j is 12   k is 102
i is 2   j is 11   k is 101
i is 2   j is 11   k is 102
i is 2   j is 12   k is 101
i is 2   j is 12   k is 102


## Exceptions

There are two approaches to error handling:

- **Look Before You Leap** (LBYL): validate variables before expression. There are a lot of error checking code before real work happens.
- **Easier to Ask Forgiveness than Permission** (EAFP): EAFP assumes code will generally work and tries to catch the occasional exceptions and handle it after they occur.

The `try/catch` framework is the latter, you try the expression and catch any errors and deal with it.

```
f = open("file")
try                                    # do this block
    # read a line
    # process a line
catch err                              # err is the exception, catch block is executed when exception is raised
    if ~isa(err, EOFError)               
        println("unexpected error: $err")
    end
finally                                # finally block is always executed
    close(f)
end
```

Here's a trivial example:

In [20]:
try
    x = -4
    log(x)
catch err
    println(">>> ", err)
end

>>> DomainError(-4.0, "log will only return a complex result if called with a complex argument. Try log(Complex(x)).")
